# 8-pixel DCR

> *This notebook computes the detectors DCR for the two 8-pixel arrays used for the 8-spot smFRET measurements*

## Load software

In [ ]:
import phconvert as phc
phc.__version__

In [ ]:
from __future__ import division
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler
%matplotlib inline
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

sns.set_style('whitegrid')
#sns.set_palette('deep')

# palette = ('Paired', 10)
# sns.palplot(sns.color_palette(*palette))
# sns.set_palette(*palette)

bmap = sns.color_palette("Set1", 9)
colors = np.array(bmap)[(1,0,2,3,4,8,6,7), :]
sns.palplot(sns.color_palette(colors))
sns.set_palette(colors)

## Load data

In [ ]:
data_dir = './data/multispot/'

In [ ]:
fname = 'Z2014-02-26_DCR_test_2.hdf5'

In [ ]:
full_fname = data_dir + fname

In [ ]:
d = phc.hdf5.load_photon_hdf5(full_fname, require_setup=False)

In [ ]:
type(d._v_file)

## Compute DCR

In [ ]:
# Compute DCR
A_em = phc.hdf5.photon_data_mapping(d._v_file, 'detectors')

In [ ]:
A_em = [a.read().view(bool) for a in A_em.values()]
accept_dcr = [a.sum()/d.acquisition_duration.read() for a in A_em]
donor_dcr = [(~a).sum()/d.acquisition_duration.read() for a in A_em]

# Make a DataFrame
dcr_data = pd.DataFrame(columns = ['Donor DCR', 'Acceptor DCR'], dtype=float)
dcr_data.index.name = 'CH'
dcr_data['Donor DCR'] = donor_dcr
dcr_data['Acceptor DCR'] = accept_dcr
dcr_data = dcr_data.round(4)
dcr_data

In [ ]:
# Output plot and tables
dcr_data.plot.bar(table=np.round(dcr_data, 2).T)
plt.ylabel('DCR (cps)')
plt.gca().xaxis.set_visible(False)

## Save results

In [ ]:
dcr_data.to_csv('results/8-pixels_DCR.csv')

In [ ]:
pd.read_csv('results/8-pixels_DCR.csv', index_col=0)

## Other

In [ ]:
from fretbursts import *

In [ ]:
def compute_dcr(d):
    # Compute DCR
    accept_dcr = [a.sum()/(d.time_max - d.time_min) for a in d.A_em]
    donor_dcr = [(~a).sum()/(d.time_max - d.time_min) for a in d.A_em]

    # Make a DataFrame
    dcr_data = pd.DataFrame(columns = ['Donor DCR', 'Acceptor DCR'])
    dcr_data['Donor DCR'] = donor_dcr
    dcr_data['Acceptor DCR'] = accept_dcr
    dcr_data.index.name = 'CH'
    return dcr_data

In [ ]:
d = loader.photon_hdf5(full_fname)

In [ ]:
d_small = d.slice_ph(time_s2=30)

In [ ]:
dcr_small = compute_dcr(d_small)
dcr_small

In [ ]:
print('Percentage DCR change:')
100*(dcr_small - dcr_data)/dcr_data

In [ ]:
print('Percentage DCR change:')
100*(dcr_small - dcr_data)/dcr_data

In [ ]:
#d.calc_bg(fun=fb.bg.raw_fit, time_s=30)

In [ ]:
#%matplotlib qt

In [ ]:
Ax = dplot(d, timetrace_single, ph_sel=Ph_sel(Dex='Aem'), binwidth=5, 
           tmax=None, show_rate_th=False, sharey=False, set_ax_limits=False)
for ax in Ax.ravel():
    ax.axvline(120)
    ax.axvline(3800)

In [ ]:
d_sel = d.slice_ph(time_s1=120, time_s2=3800)
dcr_sel = compute_dcr(d_sel)
dcr_sel

In [ ]:
dcr_sel - dcr_data

In [ ]:
d_sel.calc_bg(bg.exp_fit, time_s=30, tail_min_us='auto')

In [ ]:
raw_bg_rates_d = np.asfarray(d_sel.bg_dd).mean(1)
raw_bg_rates_a = np.asfarray(d_sel.bg_ad).mean(1)

dcr_sel['Donor DCR tail fit'] = raw_bg_rates_d
dcr_sel['Acceptor DCR tail fit'] = raw_bg_rates_a

In [ ]:
dcr_sel_d = dcr_sel[['Donor DCR', 'Donor DCR tail fit']]
dcr_sel_d.plot.bar(table=np.round(dcr_sel_d, 2).T)
plt.ylabel('DCR (cps)')
plt.gca().xaxis.set_visible(False)

In [ ]:
dcr_sel_a = dcr_sel[['Acceptor DCR', 'Acceptor DCR tail fit']]
dcr_sel_d.plot.bar(table=np.round(dcr_sel_d, 2).T)
plt.ylabel('DCR (cps)')
plt.gca().xaxis.set_visible(False)

In [ ]:
dcr_sel